In [4]:
import firebase_admin
from firebase_admin import credentials, firestore
from collections import Counter
from datetime import datetime

# Initialize Firebase Admin SDK
cred = credentials.Certificate("curious.json")


# Initialize Firestore client
db = firestore.client()

# Define the Firestore collection for events
event_collection_name = "events"

# Retrieve all documents from the collection
events_ref = db.collection(event_collection_name)
event_docs = events_ref.stream()

# Initialize data containers for events
event_data = {
    "event_count": 0,
    "status_counter": Counter(),
    "venue_counter": Counter(),
    "time_slots": {"Morning": 0, "Afternoon": 0, "Evening": 0, "Night": 0},
    "upcoming_events": [],
    "past_events": []
}

# Current date for comparison
current_date = datetime.now()

# Process each document
for doc in event_docs:
    data_dict = doc.to_dict()
    
    # Increment event count
    event_data["event_count"] += 1

    # Extract relevant fields
    status = data_dict.get('status', 'Unknown')
    date_str = data_dict.get('date', 'Unknown')
    time_str = data_dict.get('time', '00:00:00')
    title = data_dict.get('title', 'Unknown')
    venue = data_dict.get('venue', 'Unknown')

    # Update counters
    event_data["status_counter"][status] += 1
    event_data["venue_counter"][venue] += 1

    # Time slot analysis
    try:
        time = datetime.strptime(time_str, "%H:%M:%S")
        if 5 <= time.hour < 12:
            event_data["time_slots"]["Morning"] += 1
        elif 12 <= time.hour < 17:
            event_data["time_slots"]["Afternoon"] += 1
        elif 17 <= time.hour < 21:
            event_data["time_slots"]["Evening"] += 1
        else:
            event_data["time_slots"]["Night"] += 1
    except ValueError:
        print(f"Error parsing time: {time_str}. Expected format is %H:%M:%S.")

    # Date analysis
    try:
        if date_str != 'Unknown':
            event_date = datetime.strptime(date_str, "%d-%m-%Y")
            if event_date >= current_date:
                event_data["upcoming_events"].append((title, date_str, time_str, venue))
            else:
                event_data["past_events"].append((title, date_str, time_str, venue))
    except ValueError:
        print(f"Error parsing date: {date_str}. Expected format is %d-%m-%Y.")

# Define the Firestore collection for profiles
profile_collection_name = "profile"

# Retrieve all documents from the collection
profiles_ref = db.collection(profile_collection_name)
profile_docs = profiles_ref.stream()

# Initialize data containers for profiles
profile_data = {
    "college": [],
    "department": [],
    "position": []
}

# Process each document
for doc in profile_docs:
    data_dict = doc.to_dict()
    profile_data["college"].append(data_dict.get('college', 'Unknown'))
    profile_data["department"].append(data_dict.get('department', 'Unknown'))
    profile_data["position"].append(data_dict.get('position', 'Unknown'))

# Data collection complete
print("Data collection complete. Data stored in variables 'event_data' and 'profile_data'.")


Data collection complete. Data stored in variables 'event_data' and 'profile_data'.


In [5]:
event_data

{'event_count': 3,
 'status_counter': Counter({'Open': 1, 'Closed': 1, 'Ongoing': 1}),
 'venue_counter': Counter({'SRM FSH 601': 1,
          'SRM Law College': 1,
          'Ramanujar research block': 1}),
 'time_slots': {'Morning': 1, 'Afternoon': 2, 'Evening': 0, 'Night': 0},
 'upcoming_events': [],
 'past_events': [('Introduction to research methodologies',
   '19-05-2024',
   '13:12:01',
   'SRM FSH 601'),
  ('Law and order beyond the books',
   '19-05-2024',
   '16:12:01',
   'SRM Law College'),
  ('Introduction to PDE with Graphs',
   '19-05-2024',
   '10:00:43',
   'Ramanujar research block')]}

In [10]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
from bokeh.palettes import Category20
from bokeh.transform import cumsum
from math import pi
from collections import Counter

# Sample event data
event_data = {
    'event_count': 3,
    'status_counter': Counter({'Open': 1, 'Closed': 1, 'Ongoing': 1}),
    'venue_counter': Counter({'SRM FSH 601': 1, 'SRM Law College': 1, 'Ramanujar research block': 1}),
    'time_slots': {'Morning': 1, 'Afternoon': 2, 'Evening': 0, 'Night': 0},
    'upcoming_events': [],
    'past_events': [
        ('Introduction to research methodologies', '19-05-2024', '13:12:01', 'SRM FSH 601'),
        ('Law and order beyond the books', '19-05-2024', '16:12:01', 'SRM Law College'),
        ('Introduction to PDE with Graphs', '19-05-2024', '10:00:43', 'Ramanujar research block')
    ]
}

# Set output file for HTML
output_file("events.html")

# Helper function to get appropriate color palette
def get_palette(palette_name, num_colors):
    if palette_name == "Category20":
        palette = Category20[20]  # Get the full palette
        return palette[:num_colors] if num_colors <= 20 else palette
    return []

# Data preparation for visualizations
status_labels = list(event_data["status_counter"].keys())
status_values = list(event_data["status_counter"].values())
status_colors = get_palette("Category20", len(status_labels))

venue_labels = list(event_data["venue_counter"].keys())
venue_values = list(event_data["venue_counter"].values())
venue_colors = get_palette("Category20", len(venue_labels))

time_slot_labels = list(event_data["time_slots"].keys())
time_slot_values = list(event_data["time_slots"].values())
time_slot_colors = get_palette("Category20", len(time_slot_labels))

upcoming_count = len(event_data["upcoming_events"])
past_count = len(event_data["past_events"])

# Creating Bokeh plots with enhanced visuals and interactivity

# Event Status Distribution (Bar Chart)
status_source = ColumnDataSource(data=dict(labels=status_labels, counts=status_values, color=status_colors))
status_fig = figure(x_range=status_labels, height=350, title="Event Status Distribution",
                    toolbar_location=None, tools="hover", tooltips="@labels: @counts")
status_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=status_source)

status_fig.xgrid.grid_line_color = None
status_fig.y_range.start = 0
status_fig.yaxis.axis_label = 'Count'
status_fig.title.text_font_size = '16pt'
status_fig.xaxis.major_label_orientation = pi / 4
status_fig.yaxis.axis_label_standoff = 12
status_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Event Venue Distribution (Bar Chart)
venue_source = ColumnDataSource(data=dict(labels=venue_labels, counts=venue_values, color=venue_colors))
venue_fig = figure(x_range=venue_labels, height=350, title="Event Venue Distribution",
                   toolbar_location=None, tools="hover", tooltips="@labels: @counts")
venue_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=venue_source)

venue_fig.xgrid.grid_line_color = None
venue_fig.y_range.start = 0
venue_fig.yaxis.axis_label = 'Count'
venue_fig.title.text_font_size = '16pt'
venue_fig.xaxis.major_label_orientation = pi / 4
venue_fig.yaxis.axis_label_standoff = 12
venue_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Donut Plot for Event Time Slots (Pie Chart)
time_slot_angles = [(value / sum(time_slot_values)) * 2 * pi for value in time_slot_values if value > 0]
time_slot_labels = [label for label, value in zip(time_slot_labels, time_slot_values) if value > 0]
time_slot_values = [value for value in time_slot_values if value > 0]
time_slot_colors = get_palette("Category20", len(time_slot_labels))

time_slot_source = ColumnDataSource(data=dict(
    labels=time_slot_labels, counts=time_slot_values, angle=time_slot_angles, color=time_slot_colors
))
time_slot_fig = figure(height=350, title="Event Time Slots Distribution", toolbar_location=None, tools="hover", tooltips="@labels: @counts")
time_slot_fig.wedge(x=0, y=1, radius=0.4, 
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='labels', source=time_slot_source)

time_slot_fig.axis.axis_label = None
time_slot_fig.axis.visible = False
time_slot_fig.grid.grid_line_color = None
time_slot_fig.title.text_font_size = '16pt'
time_slot_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Upcoming vs Past Events (Bar Chart)
event_count_source = ColumnDataSource(data=dict(labels=["Upcoming Events", "Past Events"], counts=[upcoming_count, past_count], color=["#8073ac", "#d95f02"]))
event_count_fig = figure(x_range=["Upcoming Events", "Past Events"], height=350, title="Upcoming vs Past Events",
                         toolbar_location=None, tools="hover", tooltips="@labels: @counts")
event_count_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=event_count_source)

event_count_fig.xgrid.grid_line_color = None
event_count_fig.y_range.start = 0
event_count_fig.yaxis.axis_label = 'Count'
event_count_fig.title.text_font_size = '16pt'
event_count_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Save the plots as HTML
save(gridplot([[status_fig, venue_fig], [time_slot_fig, event_count_fig]]))


'c:\\Users\\Rajesh\\Desktop\\Workspace\\Python-Development\\cronjob\\checking\\events.html'

In [11]:
profile_data

{'college': ['', 'SRM', '', 'SRM IST', '', '', '', '', ''],
 'department': ['', 'CA', '', 'CalA', '', '', '', '', ''],
 'position': ['', 'Student', '', 'Student ', '', '', '', '', '']}

In [18]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
from bokeh.palettes import Category20, Turbo256
from bokeh.transform import linear_cmap
from bokeh.io import show
from math import pi
from collections import Counter

# Sample profile data
profile_data = {'college': ['VPT',
  'VPT',
  'SRM IT PARK',
  'SRM FSH',
  'SRM FSH',
  'SRM IT PARK',
  'Hotel Management'],
 'department': ['EEE',
  'CSE',
  'cloud computing',
  'CA',
  'CA',
  'School of Computation',
  'Hospitality'],
 'position': ['faculty',
  'research scholar',
  'faculty',
  'faculty',
  'student',
  'student',
  'Head of department']}

# Set output file for HTML
output_file("profiles_visualizations.html")

# Helper function to get appropriate color palette
def get_palette(palette_name, num_colors):
    if palette_name == "Category20":
        palette = Category20[20]  # Get the full palette
        return palette[:num_colors] if num_colors <= 20 else palette
    elif palette_name == "Turbo256":
        return Turbo256[:num_colors] if num_colors <= len(Turbo256) else Turbo256
    return []

# Data preparation for visualizations
def prepare_data_for_plot(data_list):
    counts = Counter(data_list)
    labels = list(counts.keys())
    values = list(counts.values())
    return labels, values

# Prepare data for each visualization
college_labels, college_values = prepare_data_for_plot(profile_data["college"])
department_labels, department_values = prepare_data_for_plot(profile_data["department"])
position_labels, position_values = prepare_data_for_plot(profile_data["position"])

# Create Bokeh plots

# College Distribution (Bar Chart)
college_colors = get_palette("Category20", len(college_labels))
college_source = ColumnDataSource(data=dict(labels=college_labels, counts=college_values, color=college_colors))
college_fig = figure(x_range=college_labels, height=350, title="Profile Distribution by College",
                     toolbar_location=None, tools="hover", tooltips="@labels: @counts")
college_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=college_source)

college_fig.xgrid.grid_line_color = None
college_fig.y_range.start = 0
college_fig.yaxis.axis_label = 'Count'
college_fig.title.text_font_size = '16pt'
college_fig.xaxis.major_label_orientation = pi / 4
college_fig.yaxis.axis_label_standoff = 12
college_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Department Distribution (Bar Chart)
department_colors = get_palette("Category20", len(department_labels))
department_source = ColumnDataSource(data=dict(labels=department_labels, counts=department_values, color=department_colors))
department_fig = figure(x_range=department_labels, height=350, title="Profile Distribution by Department",
                        toolbar_location=None, tools="hover", tooltips="@labels: @counts")
department_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=department_source)

department_fig.xgrid.grid_line_color = None
department_fig.y_range.start = 0
department_fig.yaxis.axis_label = 'Count'
department_fig.title.text_font_size = '16pt'
department_fig.xaxis.major_label_orientation = pi / 4
department_fig.yaxis.axis_label_standoff = 12
department_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Position Distribution (Bar Chart)
position_colors = get_palette("Category20", len(position_labels))
position_source = ColumnDataSource(data=dict(labels=position_labels, counts=position_values, color=position_colors))
position_fig = figure(x_range=position_labels, height=350, title="Profile Distribution by Position",
                      toolbar_location=None, tools="hover", tooltips="@labels: @counts")
position_fig.vbar(x='labels', top='counts', width=0.9, color='color', source=position_source)

position_fig.xgrid.grid_line_color = None
position_fig.y_range.start = 0
position_fig.yaxis.axis_label = 'Count'
position_fig.title.text_font_size = '16pt'
position_fig.xaxis.major_label_orientation = pi / 4
position_fig.yaxis.axis_label_standoff = 12
position_fig.add_tools(HoverTool(tooltips="@labels: @counts"))

# Arrange plots in a layout
layout = gridplot([[college_fig, department_fig], [position_fig]])

# Save the plots as HTML
save(layout)


'c:\\Users\\Rajesh\\Desktop\\Workspace\\Python-Development\\cronjob\\checking\\profiles_visualizations.html'